In [302]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline
import os
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import KMeans, DBSCAN

Подключаем необходимые библиотеки.

In [303]:
os.chdir(r'C:\Users\ммм\Desktop\CSC\Анализ данных\Lect_04\задача_1_леденцы')

Задаем рабочую папку.

In [304]:
dataset = pd.read_csv("ledency.dat", sep=";")
datacopy = dataset.copy(deep=True)

Импортируем наши данные. Делаем копию данных, для удобной работы с ними.

In [305]:
dataset.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
0,4,5,5,5,3,3,3,3,3,3,3
1,5,4,5,5,3,3,3,3,3,3,3
2,5,5,4,5,3,3,3,3,3,3,3
3,5,4,5,5,3,3,3,3,3,3,3
4,4,5,5,5,3,3,3,3,3,3,3


In [306]:
print(dataset)

     V1  V2  V3  V4  V5  V6  V7  V8  V9  V10  V11
0     4   5   5   5   3   3   3   3   3    3    3
1     5   4   5   5   3   3   3   3   3    3    3
2     5   5   4   5   3   3   3   3   3    3    3
3     5   4   5   5   3   3   3   3   3    3    3
4     4   5   5   5   3   3   3   3   3    3    3
..   ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...
333   3   3   3   3   3   3   5   5   3    3    3
334   3   3   3   3   3   3   5   5   3    3    3
335   3   3   3   3   3   3   5   5   3    3    3
336   3   3   3   3   3   3   5   5   3    3    3
337   3   3   3   3   3   3   5   5   3    3    3

[338 rows x 11 columns]


Понимаем размерность данных, с которыми мы работаем - 338 человек ответили в опросе по 11 пунктам.

In [307]:
dataset.describe(include='all')

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
count,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000,338.000000
mean,3.544379,3.526627,3.517751,3.411243,3.207101,3.204142,3.254438,3.289941,3.603550,3.550296,3.565089
std,0.955748,0.956410,0.990150,0.992311,0.867314,0.869725,0.844107,0.821784,0.969682,0.927082,0.942042
min,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
25%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
50%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,5.000000,5.000000,5.000000,4.000000,3.000000,3.000000,3.000000,3.000000,5.000000,5.000000,5.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


Убеждаемся, что с данными все порядке: мы работаем с количественными переменными, каждый из 338 человек ответил на все из 11 вопросов, а также замечаем, что минимум во всех столбцах - 2, а максимум - 5, поэтому опрашиваемые ставили оценки в данном опросе от 2 до 5, следовательно стандартизация не нужна.

Как это было на лекции, постараемся итеративно, эмпирически найти параметры epsilon и m такие, чтобы кластеризация была адекватной.

In [308]:
dbs = DBSCAN(eps=0.5, metric='euclidean', min_samples=5)
dbs.fit(dataset)
dbs.labels_
datacopy['dbscan'] = dbs.labels_
datacopy.groupby('dbscan').size()

dbscan
-1     60
 0     10
 1      5
 2     49
 3     12
 4      9
 5     49
 6     15
 7     10
 8      6
 9     37
 10    10
 11     5
 12     9
 13    52
dtype: int64

Наблюдения в кластерах расположены неравномерно, а также кластеров много, учитывая то что есть малочисленные кластеры. Это плохо - увеличиваем eplison и m.

In [309]:
dbs = DBSCAN(eps=1.5, metric='euclidean', min_samples=8)
dbs.fit(dataset)
dbs.labels_
datacopy['dbscan'] = dbs.labels_
datacopy.groupby('dbscan').size()

dbscan
-1     29
 0    104
 1     15
 2     59
 3     16
 4     46
 5     15
 6     54
dtype: int64

Опять, наблюдения в кластерах расположены неравномерно, увеличиваем eplison и m.

In [310]:
dbs = DBSCAN(eps=1.8, metric='euclidean', min_samples=11)
dbs.fit(dataset)
dbs.labels_
datacopy['dbscan'] = dbs.labels_
datacopy.groupby('dbscan').size()

dbscan
-1     10
 0    110
 1    100
 2     61
 3     57
dtype: int64

Выглядит более адекватно, но в таком случае 10 наблюдений классифицировались как выбросы, попробуем уменьшить m.

In [311]:
dbs = DBSCAN(eps=1.8, metric='euclidean', min_samples=4)
dbs.fit(dataset)
dbs.labels_
datacopy['dbscan'] = dbs.labels_
datacopy.groupby('dbscan').size()

dbscan
-1      2
 0    110
 1    104
 2      4
 3     61
 4     57
dtype: int64

Кластеризация выглядит адекватной. Будем продолжать работать с версией кластеризации, когда у нас espilon = 1.8, m = 4. Посмотрим средние по группам кластеров, а также посмотрим, вручную, какие наблюдения попали в -1 кластер (выбросы) и во 2 кластер - может быть их следует поместить в какой-либо другой кластер. 

In [312]:
dataset['dbscan'] = dbs.labels_
dataset.groupby('dbscan').mean()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
dbscan,,,,,,,,,,,
-1,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,5.000000,5.000000,3.000000,3.000000,3.000000
0,4.845455,4.800000,4.836364,4.681818,3.036364,3.036364,3.000000,2.990909,3.072727,3.045455,3.045455
1,2.855769,2.788462,2.721154,2.567308,2.634615,2.605769,2.778846,2.913462,4.951923,4.817308,4.865385
2,2.000000,4.000000,4.000000,4.000000,2.000000,2.000000,2.000000,3.000000,5.000000,4.750000,4.750000
3,3.000000,3.000000,3.000000,3.000000,4.852459,4.852459,3.000000,2.918033,2.754098,2.754098,2.754098
4,3.000000,3.000000,3.000000,2.947368,2.947368,2.947368,4.912281,4.912281,3.000000,3.000000,3.000000


In [313]:
dataset[dataset['dbscan'].isin([-1])]

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,dbscan
308,3,2,2,2,2,3,5,5,3,3,3,-1
324,3,2,2,2,2,3,5,5,3,3,3,-1


In [314]:
dataset[dataset['dbscan'].isin([2])]

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,dbscan
112,2,4,4,4,2,2,2,3,5,5,5,2
139,2,4,4,4,2,2,2,3,5,5,5,2
177,2,4,4,4,2,2,2,3,5,4,4,2
188,2,4,4,4,2,2,2,3,5,5,5,2


Видим, что наблюдения, попавшие в -1 группу (выбросы), следовало бы отнести в 4 группу (потому что средние по этим группам различаются незначительно и интерпритация этих групп одинаковая), а наблюдения, попавшие во 2 группу, следовало бы отнести в 1 группу (потому что интерпритация этих групп одинаковая).

In [315]:
for i in range(len(dbs.labels_)):
    if dbs.labels_[i] == -1:
        dbs.labels_[i] = 4
    elif dbs.labels_[i] == 2:
        dbs.labels_[i] = 1

Тогда имеем следующие 4 кластера.

In [316]:
dataset['dbscan'] = dbs.labels_
dataset.groupby('dbscan').mean()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
dbscan,,,,,,,,,,,
0,4.845455,4.800000,4.836364,4.681818,3.036364,3.036364,3.000000,2.990909,3.072727,3.045455,3.045455
1,2.824074,2.833333,2.768519,2.620370,2.611111,2.583333,2.750000,2.916667,4.953704,4.814815,4.861111
3,3.000000,3.000000,3.000000,3.000000,4.852459,4.852459,3.000000,2.918033,2.754098,2.754098,2.754098
4,3.000000,2.966102,2.966102,2.915254,2.915254,2.949153,4.915254,4.915254,3.000000,3.000000,3.000000


Получившиеся четыре группы можно проинтерпретировать следующим образом:

0 группа - те, кто поставили максимальные оценки по 1, 2, 3 и 4 пунктам - это те, кто потребляет леденцы для того, чтобы освежить полость рта, избавиться от неприятного запаха, привкуса во рту и т.д.

1 группа - те кто поставили максимальные оценки по 9, 10 и 11 пунктам - это те, кто потребляет леденцы для лечения, для улучшения состояния горла, носа и т.д.

2 группа - те, кто поставили максимальные оценки по 5 и 6 пунктам - это такие "сладкоежки", которые потребляют леденцы как источник/замена чего-то сладкого

4 группа - те, кто поставили максимальные оценки по 7 и 8 пунктам - это те, кто потребляет леденцы для концентрации внимания, отвлечения и т.д.

Размеры кластеров следующие:

In [317]:
dataset.groupby('dbscan').size()

dbscan
0    110
1    108
3     61
4     59
dtype: int64

Номера наблюдений, попавших в нулевой (первая группа) кластер.

In [318]:
dataset[dataset['dbscan']==0].index.values

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109], dtype=int64)

Номера наблюдений, попавших в первый (вторая группа) кластер.

In [319]:
dataset[dataset['dbscan']==1].index.values

array([110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174,
       175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187,
       188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200,
       201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213,
       214, 215, 216, 217], dtype=int64)

Номера наблюдений, попавших в третий (третья группа) кластер.

In [320]:
dataset[dataset['dbscan']==3].index.values

array([218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230,
       231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243,
       244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256,
       257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269,
       270, 271, 272, 273, 274, 275, 276, 277, 278], dtype=int64)

Номера наблюдений, попавших в четвертый (четвертая группа) кластер.

In [321]:
dataset[dataset['dbscan']==4].index.values

array([279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291,
       292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304,
       305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317,
       318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330,
       331, 332, 333, 334, 335, 336, 337], dtype=int64)

Сравним полученные результаты с результатами иерархического кластерного анализа и анализа методом k-средних.

In [322]:
link = linkage(dataset, 'ward', 'euclidean')
dataset['clusterhierarcical'] = fcluster(link, 21.65, criterion='distance')
dataset.groupby('clusterhierarcical').size()
km = KMeans(n_clusters=4, random_state=54)
km.fit(dataset)
dataset['clusterkmeans'] = km.labels_

In [323]:
dataset.groupby('clusterhierarcical').size()

clusterhierarcical
1    110
2    108
3     59
4     61
dtype: int64

In [324]:
dataset.groupby('clusterkmeans').size()

clusterkmeans
0    108
1    110
2     61
3     59
dtype: int64

In [325]:
dataset.groupby('dbscan').size()

dbscan
0    110
1    108
3     61
4     59
dtype: int64

Видим, что размеры полученных групп во всех трех методах кластеризации одинаковые. Посмотрим, одинаковы ли сами кластеры.

In [326]:
tab = pd.crosstab(dataset['clusterhierarcical'], dataset['dbscan'])
print(tab)

dbscan                0    1   3   4
clusterhierarcical                  
1                   110    0   0   0
2                     0  108   0   0
3                     0    0   0  59
4                     0    0  61   0


In [327]:
tab = pd.crosstab(dataset['clusterkmeans'], dataset['dbscan'])
print(tab)

dbscan           0    1   3   4
clusterkmeans                  
0                0  108   0   0
1              110    0   0   0
2                0    0  61   0
3                0    0   0  59


In [328]:
tab = pd.crosstab(dataset['clusterkmeans'], dataset['clusterhierarcical'])
print(tab)

clusterhierarcical    1    2   3   4
clusterkmeans                       
0                     0  108   0   0
1                   110    0   0   0
2                     0    0   0  61
3                     0    0  59   0


Из таких попарных пересечений (в терминах общих наблюдений) различных номеров групп потребителей (при иерархическом кластерном анализе и кластерном анализе методом DBSCAN и при кластеризации методом k-средних и кластерном анализе методом DBSCAN) можно сделать вывод, что все три кластеризации (иерархическим кластерным анализом, кластерным анализом методом k-средних и методом DBSCAN) в точности совпали - различия только в различной нумерации групп потребителей.

Вывод. Таким образом, в ходе выполнения лабораторной работы в результате кластеризации методом DBSCAN были выделены четыре следующих группы потребителей леденцов:

1) Первая группа потребителей - те, кто потребляет леденцы для того, чтобы избавиться от заложенности в носу/горле.

2) Вторая группа потребителей - те, кто потребляет леденцы для того, чтобы получить "второе дыхаение", чувствовать себя бодрее.

3) Третья группа потребителей - "сладкоежки", которые потребляют леденцы как что-то сладкое/заменитель чего-то сладкого.

4) Четвертая группа потребителей - те, кто потребляет леденцы для увеличения концентрации и для того, чтобы сосредоточиться.

Также было выяснено, что результаты кластеризации исходных данных методом DBSCAN, методом k-средних и методом иерархического кластерного анализа в точности совпадают - различия могут быть только в различной нумерации групп потребителей.